# Links
- [Drive Artigos](https://drive.google.com/drive/folders/1R9LVdajgDMxF-zba9aMeEOeSs8wfhzU6?hl=pt-br)
- [Google Notebook LLM](https://notebooklm.google.com/notebook/8c75ea8a-3ce1-4cfd-bbc5-9819d95599e1)

# Larissa

In [1]:
import shutil
import cv2
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import time
import csv

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


True
4
NVIDIA H100 80GB HBM3


In [2]:

# le2i = "/content/drive/MyDrive/TCC2/le2i" #videos
# URFD = "/content/drive/MyDrive/TCC2/URFD" # imgs
fall = "/fall_dataset" #imgs YOLO labeled

# listar arquivos da pasta
# os.listdir(le2i)
# os.listdir(URFD)
# os.listdir(fall)

##Usando Fall Dataset

In [2]:
# funcoes para rodar experimentos 

def format_time(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}h {int(minutes):02d}m {seconds:.2f}s"

def run_experiment(nome_modelo, nome_yaml, nome_experimento):

    NOME_MODELO = nome_modelo
    NOME_YAML = nome_yaml
    NOME_EXPERIMENTO = nome_experimento

    start_training_time = time.perf_counter()

    model = YOLO(NOME_MODELO)
    model.train(data=NOME_YAML, batch=8, epochs=100, device=0, name=NOME_EXPERIMENTO)

    end_training_time = time.perf_counter()
    training_duration = end_training_time - start_training_time

    best_model_path = model.trainer.best
    start_validation_time = time.perf_counter()

    best_model = YOLO(best_model_path)
    results = best_model.val()

    end_validation_time = time.perf_counter()
    validation_duration = end_validation_time - start_validation_time

    training_duration_str = format_time(training_duration)
    validation_duration_str = format_time(validation_duration)
    map50_95 = results.box.map
    map50 = results.box.map50
    precision = results.box.mp
    recall = results.box.mr

    print("\n--- Relatório Final de Execução ---")
    print(f"Tempo de Treinamento..: {training_duration_str}")
    print(f"Tempo de Validação....: {validation_duration_str}")
    print("--- Métricas de Performance ---")
    print(f"mAP50-95 (principal)..: {map50_95:.4f}")
    print(f"mAP50 (IoU > 0.5).....: {map50:.4f}")
    print(f"Precisão (Precision)..: {precision:.4f}")
    print(f"Recall................: {recall:.4f}")

    header = [
        'experimento', 'dataset', 'modelo', 'tempo_treino', 'tempo_validacao',
        'mAP50_95', 'mAP50', 'precisao', 'recall'
    ]
    data_row = [
        NOME_EXPERIMENTO, NOME_YAML, NOME_MODELO, training_duration_str, validation_duration_str,
        f"{map50_95:.4f}", f"{map50:.4f}", f"{precision:.4f}", f"{recall:.4f}"
    ]

    nome_csv = 'resultados.csv'
    file_exists = os.path.isfile(nome_csv)
    with open(nome_csv, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists:
            writer.writerow(header)
        writer.writerow(data_row)

    print(f"\nResultados foram salvos em '{nome_csv}'")




In [ ]:
# Holdout 70/30

run_experiment('yolo11n.pt', './yaml/fall-data7030.yaml', 'split_7030_run1')

Ultralytics 8.3.206 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA H100 80GB HBM3, 81090MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=fall-data7030.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=split_7030_run14, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T

In [ ]:
# Holdout 60/40

run_experiment('yolo11n.pt', './yaml/fall-data6040.yaml', 'split_6040_run1')

Ultralytics 8.3.206 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA H100 80GB HBM3, 81090MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=fall-data6040.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=split_6040_run12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T

In [ ]:
# Holdout 80/20

run_experiment('yolo11n.pt', './yaml/fall-data8020.yaml', 'split_8020_run1')

Ultralytics 8.3.206 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA H100 80GB HBM3, 81090MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=fall-data8020.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=split_8020_run12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T